<a href="https://colab.research.google.com/github/RichardCarrionF/RichardCarrionF/blob/main/FIFA2026_ClimateVolatility_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
# ============================================================
# From Carbon Neutrality to Climate Resilience:
# Python Analysis for FIFA 2026 Host Cities
# Richard Carrión | Python for Public Policy

In [29]:
# ---------- Imports ----------
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler

In [35]:
CSV_FILE = "GlobalLandTemperaturesByCity.csv"


In [23]:
# Windows used in comparisons
EARLY_START, EARLY_END   = 1900, 1920
RECENT_START, RECENT_END = 2000, 2020

In [24]:
# Baseline window for flagging "extreme" years (> mean + 1 SD)
BASELINE_START, BASELINE_END = 1900, 1950

In [25]:
# Optional warming context (first vs recent decades)
WARM_EARLY_START,  WARM_EARLY_END  = 1900, 1910
WARM_RECENT_START, WARM_RECENT_END = 2010, 2020

In [26]:
# FIFA 2026 host cities as they appear in the dataset
DATASET_CITIES = [
    "Atlanta","Boston","Dallas","Houston","Kansas City","Los Angeles","Miami",
    "New York","Philadelphia","San Francisco","Seattle","Toronto","Vancouver",
    "Mexico City","Guadalajara","Monterrey"
]

In [27]:
# ---------- Small helpers ----------
def c_to_f(x):
    """Convert Celsius to Fahrenheit for easier policy communication."""
    return x * 9/5 + 32

def rolling_sd(s, w=ROLL_YEARS):
    """Rolling standard deviation = our variability measure."""
    return s.rolling(w, min_periods=3).std()

def rolling_mean(s, w=ROLL_YEARS):
    """Centered rolling mean for smoother trends in plots."""
    return s.rolling(w, min_periods=3, center=True).mean()

In [37]:
# 1) Load & clean
# ============================================================
df = pd.read_csv(CSV_FILE)
df["dt"] = pd.to_datetime(df["dt"], errors="coerce")
df = df.dropna(subset=["dt", "AverageTemperature"])
df["Year"]  = df["dt"].dt.year
df["Month"] = df["dt"].dt.month
df = df[df["Year"] >= START_YEAR]

# Keep only the 16 World Cup 2026 host cities
df = df[df["City"].isin(DATASET_CITIES)].copy()

In [38]:
# 2) Annual analysis (all months)
#    - annual mean temp (°F)
#    - 5-yr rolling SD (variability)
#    - Δ variability (recent vs early)
#    - extreme-year counts
#    - CVI (0–1)
# ============================================================
# Aggregate monthly → annual per city
annual = (df.groupby(["Country","City","Year"], as_index=False)
            .agg(AvgTemp_C=("AverageTemperature","mean"),
                 Unc_C=("AverageTemperatureUncertainty","mean")))
annual["AvgTemp_F"] = c_to_f(annual["AvgTemp_C"])
annual["Unc_F"]     = c_to_f(annual["Unc_C"])
annual = annual.sort_values(["City","Year"])

# Variability and smoothed trend for visuals
annual["Temp_SD_5yr"]   = annual.groupby("City")["AvgTemp_F"].transform(rolling_sd)
annual["AvgTemp_F_5yr"] = annual.groupby("City")["AvgTemp_F"].transform(rolling_mean)

In [39]:
# Δ variability: compare early 1900–1920 vs recent 2000–2020
early_mask  = annual["Year"].between(EARLY_START, EARLY_END)
recent_mask = annual["Year"].between(RECENT_START, RECENT_END)
early_var = (annual[early_mask].groupby("City", as_index=False)["Temp_SD_5yr"]
             .mean().rename(columns={"Temp_SD_5yr":"Early_SD"}))
recent_var = (annual[recent_mask].groupby("City", as_index=False)["Temp_SD_5yr"]
              .mean().rename(columns={"Temp_SD_5yr":"Recent_SD"}))
var_change = (early_var.merge(recent_var, on="City", how="inner")
                       .merge(annual[["City","Country"]].drop_duplicates(), on="City", how="left"))
var_change["Change_in_Variability"] = var_change["Recent_SD"] - var_change["Early_SD"]

In [40]:
# Extreme years: flag each year above baseline mean + 1 SD
baseline = annual[annual["Year"].between(BASELINE_START, BASELINE_END)]
base_stats = (baseline.groupby("City", as_index=False)
              .agg(Base_Mean=("AvgTemp_F","mean"), Base_SD=("AvgTemp_F","std")))
annual = annual.merge(base_stats, on="City", how="left")
annual["Extreme_Year"] = annual["AvgTemp_F"] > (annual["Base_Mean"] + annual["Base_SD"])
extreme_counts = (annual.groupby(["City","Country"], as_index=False)["Extreme_Year"]
                  .sum().rename(columns={"Extreme_Year":"Num_Extreme_Years"}))

In [41]:
# Optional: simple warming delta for narration (first vs recent decade)
warm_early  = (annual[annual["Year"].between(WARM_EARLY_START, WARM_EARLY_END)]
               .groupby("City", as_index=False)["AvgTemp_F"].mean()
               .rename(columns={"AvgTemp_F":"Early_AvgTemp_F"}))
warm_recent = (annual[annual["Year"].between(WARM_RECENT_START, WARM_RECENT_END)]
               .groupby("City", as_index=False)["AvgTemp_F"].mean()
               .rename(columns={"AvgTemp_F":"Recent_AvgTemp_F"}))
warm_delta = (warm_early.merge(warm_recent, on="City", how="inner")
                        .merge(annual[["City","Country"]].drop_duplicates(), on="City", how="left"))
warm_delta["Delta_F"] = warm_delta["Recent_AvgTemp_F"] - warm_delta["Early_AvgTemp_F"]

In [42]:
# CVI: average of normalized Δ variability and normalized extreme-year count (0–1)
cvi_data = var_change[["City","Country","Change_in_Variability"]].merge(
    extreme_counts[["City","Num_Extreme_Years"]], on="City", how="left").fillna(0)
scaler = MinMaxScaler()
cvi_data[["Norm_Var","Norm_Extreme"]] = scaler.fit_transform(
    cvi_data[["Change_in_Variability","Num_Extreme_Years"]])
cvi_data["CVI"] = (cvi_data["Norm_Var"] + cvi_data["Norm_Extreme"]) / 2

In [43]:
# 3) June–July analysis (World Cup window)
#    Repeat the same logic using only months 6 & 7.
# ============================================================
df_jj = df[df["Month"].isin([6,7])].copy()

annual_jj = (df_jj.groupby(["Country","City","Year"], as_index=False)
                .agg(AvgTemp_C=("AverageTemperature","mean"),
                     Unc_C=("AverageTemperatureUncertainty","mean")))
annual_jj["AvgTemp_F"] = c_to_f(annual_jj["AvgTemp_C"])
annual_jj["Unc_F"]     = c_to_f(annual_jj["Unc_C"])
annual_jj = annual_jj.sort_values(["City","Year"])

annual_jj["Temp_SD_5yr"]   = annual_jj.groupby("City")["AvgTemp_F"].transform(rolling_sd)
annual_jj["AvgTemp_F_5yr"] = annual_jj.groupby("City")["AvgTemp_F"].transform(rolling_mean)

early_mask_jj  = annual_jj["Year"].between(EARLY_START, EARLY_END)
recent_mask_jj = annual_jj["Year"].between(RECENT_START, RECENT_END)
early_var_jj = (annual_jj[early_mask_jj].groupby("City", as_index=False)["Temp_SD_5yr"]
                .mean().rename(columns={"Temp_SD_5yr":"Early_SD"}))
recent_var_jj = (annual_jj[recent_mask_jj].groupby("City", as_index=False)["Temp_SD_5yr"]
                 .mean().rename(columns={"Temp_SD_5yr":"Recent_SD"}))
var_change_jj = (early_var_jj.merge(recent_var_jj, on="City", how="inner")
                             .merge(annual_jj[["City","Country"]].drop_duplicates(), on="City", how="left"))
var_change_jj["Change_in_Variability"] = var_change_jj["Recent_SD"] - var_change_jj["Early_SD"]

baseline_jj = annual_jj[annual_jj["Year"].between(BASELINE_START, BASELINE_END)]
base_stats_jj = (baseline_jj.groupby("City", as_index=False)
                 .agg(Base_Mean=("AvgTemp_F","mean"), Base_SD=("AvgTemp_F","std")))
annual_jj = annual_jj.merge(base_stats_jj, on="City", how="left")
annual_jj["Extreme_Year"] = annual_jj["AvgTemp_F"] > (annual_jj["Base_Mean"] + annual_jj["Base_SD"])
extreme_counts_jj = (annual_jj.groupby(["City","Country"], as_index=False)["Extreme_Year"]
                     .sum().rename(columns={"Extreme_Year":"Num_Extreme_Years"}))

cvi_jj = var_change_jj[["City","Country","Change_in_Variability"]].merge(
    extreme_counts_jj[["City","Num_Extreme_Years"]], on="City", how="left").fillna(0)
scaler_jj = MinMaxScaler()
cvi_jj[["Norm_Var","Norm_Extreme"]] = scaler_jj.fit_transform(
    cvi_jj[["Change_in_Variability","Num_Extreme_Years"]])
cvi_jj["CVI"] = (cvi_jj["Norm_Var"] + cvi_jj["Norm_Extreme"]) / 2

In [46]:
# 4) Visuals (Figure 1–4)
# ============================================================
fig1 = px.line(
    annual, x="Year", y="AvgTemp_F_5yr", color="City",
    title="Figure 1 – Annual Temperature Trends (5-year smoothed)",
    labels={"AvgTemp_F_5yr":"Temperature (°F)"}
)
fig1.update_layout(template="plotly_white")
fig1.update_traces(line=dict(width=1), opacity=0.5)
fig1.show()

fig2 = px.bar(
    var_change.sort_values("Change_in_Variability", ascending=False),
    x="City", y="Change_in_Variability", color="Country",
    title=f"Figure 2 – Change in Variability (5-year SD) {EARLY_START}-{EARLY_END} → {RECENT_START}-{RECENT_END}",
    labels={"Change_in_Variability":"Δ 5-year SD (°F)"},
    text_auto=".2f"
)
fig2.update_layout(template="plotly_white")
fig2.show()

fig3 = px.bar(
    cvi_data.sort_values("CVI", ascending=False),
    x="City", y="CVI", color="CVI", color_continuous_scale="Reds",
    title="Figure 3 – Climate Volatility Index (CVI), Annual (0–1)",
    labels={"CVI":"Volatility (0–1)"},
    text_auto=".2f"
)
fig3.update_layout(template="plotly_white", showlegend=False)
fig3.show()

fig4 = px.bar(
    cvi_jj.sort_values("CVI", ascending=False),
    x="City", y="CVI", color="CVI", color_continuous_scale="Reds",
    title="Figure 4 – Climate Volatility Index (CVI), June–July (0–1)",
    labels={"CVI":"Volatility (0–1)"},
    text_auto=".2f"
)
fig4.update_layout(template="plotly_white", showlegend=False)
fig4.show()

In [45]:
# 5) Exports (tidy CSVs for reproducibility / dashboards)
# ============================================================
annual.to_csv("fifa2026_annual_with_variability.csv", index=False)
var_change.to_csv("fifa2026_variability_change.csv", index=False)
extreme_counts.to_csv("fifa2026_extreme_year_counts.csv", index=False)
cvi_data.to_csv("fifa2026_climate_volatility_index.csv", index=False)

annual_jj.to_csv("fifa2026_JuneJuly_annual.csv", index=False)
var_change_jj.to_csv("fifa2026_JuneJuly_variability_change.csv", index=False)
extreme_counts_jj.to_csv("fifa2026_JuneJuly_extreme_year_counts.csv", index=False)
cvi_jj.to_csv("fifa2026_JuneJuly_CVI.csv", index=False)

print("Done. Figures 1–4 rendered and CSVs saved.")

Done. Figures 1–4 rendered and CSVs saved.
